In [5]:
import pyscf
from pyscf import mcscf
import scipy.sparse
import openfermion as of
import numpy as np
from openfermionpyscf._run_pyscf import run_pyscf
from force_utility import gradient_mo_operator, get_lih, get_hf_fd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# The Hellman-Feynman force operator
The Hellman-Feynman can be found as the derivative of the Hamiltonian matrix elements with respect to the position. We will use the example of $LiH$ and obtain the operator through finite differences and analytical derivation. 

In [6]:
coords = [0, 0, 0, 0, 0, 1.4]
#coords = [0, 0, 0, 0, 1.4, 1.4]/np.sqrt(2)
delta = 1e-5
occ = None
act= None
mol = get_lih(coords,'sto-3g')
h = of.transforms.jordan_wigner(mol.get_molecular_hamiltonian(occupied_indices=occ,active_indices=act))
eig, eigv = scipy.sparse.linalg.eigsh(of.get_sparse_operator(h), k=1, which='SA') 
f = get_hf_fd(coords, delta, 'sto-3g', occupied=occ, active=act)
print("FCI energy is {}".format(eig[0]))

converged SCF energy = -7.8605386610207
converged SCF energy = -7.8605386610207
converged SCF energy = -7.8605386610207
converged SCF energy = -7.8605386610207
converged SCF energy = -7.8605386610207
converged SCF energy = -7.8605386610207
FCI energy is -7.878453652277172


What can you tell about the difference between the number of terms in the Hamiltonian and forces? How many Pauli-words do they have in common? What does that mean for the measurements?

In [7]:
print("Number of pauli-words in the hamiltonian: {}".format(len(h.terms)))
for force in f:
    print("Number of pauli-words in the force: {}".format(len(force.terms)))
    common = 0
    for key in force.terms.keys():
        if key in h.terms.keys():
            common += 1
    print("Number of pauli-words in common with hamiltonian {}".format(common))

Number of pauli-words in the hamiltonian: 631
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 621
Number of pauli-words in common with hamiltonian 621
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 621
Number of pauli-words in common with hamiltonian 621


We can now evaluate this operator with respect to the FCI state. What do you expect for the values of the forces? Will they be equal of different for the different atoms? What with the acceleration?

In [8]:
h_eval= of.expectation(of.get_sparse_operator(h), eigv)
f_eval=[]
for force in f:
    if len(force.terms)==0:
        f_eval.append(0.)
    else:
        f_eval.append(of.expectation(of.get_sparse_operator(force), eigv))
print("Hamiltonian expectation: {}, Hamiltonian eigenvalue: {}".format(h_eval,eig[0]))
print(f_eval)
f_eval = [val.real for val in f_eval]

Hamiltonian expectation: (-7.87845365227716+4.440892098500626e-16j), Hamiltonian eigenvalue: -7.878453652277172
[(8.705475287582261e-16-1.232595164407831e-31j), (-7.505472048578799e-16+0j), (-0.011280097482221882+8.673617379884035e-19j), (-8.705475287582253e-16+9.860761315262648e-32j), (7.505472048578793e-16+1.1832913578315177e-30j), (0.011280097367202285-8.673617379884035e-19j)]


Rotate the geometry by 45 degrees and repeat the excercise. What do you see?

# Force operators
We will now evaluate the force operators through the analytical formulas both with and without Pulay terms.

In [9]:
oei, tei = mol.get_integrals()
mymol = pyscf.gto.Mole(atom=[['Li', tuple(coords[0:3])],
                           ['H', tuple(coords[3:6])]],
                     basis = 'sto-3g')
mymol.build()
mf = pyscf.scf.RHF(mymol)
mf.kernel()
f_op = gradient_mo_operator(mymol, mf.mo_coeff, oei, tei)
f_op = [of.transforms.jordan_wigner(force) for force in f_op]
f_op_hf = gradient_mo_operator(mymol, mf.mo_coeff, oei, tei, with_pulay=False)
f_op_hf = [of.transforms.jordan_wigner(force) for force in f_op_hf]


converged SCF energy = -7.8605386610207


We can repeat the same analysis as before to see which terms are shared with the Hamiltonian. What do you notice? What does this mean for the cost of evaluation?

Is the Hellman-Feynman operator consistent with the finite differences?

In [10]:
print("Number of pauli-words in the hamiltonian: {}".format(len(h.terms)))
for force in f_op_hf:
    print("Number of pauli-words in the force: {}".format(len(force.terms)))
    common = 0
    large = 0
    for key in force.terms.keys():
        if key in h.terms.keys():
            common += 1
    print("Number of pauli-words in common with hamiltonian {}".format(common))

Number of pauli-words in the hamiltonian: 631
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 621
Number of pauli-words in common with hamiltonian 621
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 621
Number of pauli-words in common with hamiltonian 621


In [11]:
print("Number of pauli-words in the hamiltonian: {}".format(len(h.terms)))
for force in f_op:
    print("Number of pauli-words in the force: {}".format(len(force.terms)))
    common = 0
    for key in force.terms.keys():
        if key in h.terms.keys():
            common += 1
    print("Number of pauli-words in common with hamiltonian {}".format(common))

Number of pauli-words in the hamiltonian: 631
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 621
Number of pauli-words in common with hamiltonian 621
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 897
Number of pauli-words in common with hamiltonian 1
Number of pauli-words in the force: 621
Number of pauli-words in common with hamiltonian 621


We can now evaluate the force operators with the FCI state.

In [12]:
f_op_hf_eval=[]
for force in f_op_hf:
    if len(force.terms)==0:
        f_op_hf_eval.append(0.)
    else:
        f_op_hf_eval.append(of.expectation(of.get_sparse_operator(force), eigv))
print(f_op_hf_eval)
f_op_hf_eval = [val.real if np.abs(val) > 1e-8 else 0 for val in f_op_hf_eval]

[(4.60673913253544e-16-3.0814879110195774e-32j), (-3.97172476540298e-16-1.9721522630525295e-31j), (-0.0059691705428689245-4.336808689942018e-19j), (-4.606739132535437e-16-7.549645381997965e-32j), (3.97172476540298e-16+1.9721522630525295e-31j), (0.0059691705428693235+0j)]


In [13]:
f_op_eval=[]
for force in f_op:
    if len(force.terms)==0:
        f_op_eval.append(0.)
    else:
        f_op_eval.append(of.expectation(of.get_sparse_operator(force), eigv))
print(f_op_eval)
f_op_eval = [val.real if np.abs(val) > 1e-8 else 0 for val in f_op_eval]

[(3.3915389167665497e-16+1.232595164407831e-32j), (-2.9744704982847267e-16+0j), (0.03350041368950822+0j), (-3.391538916766543e-16+1.494521636844495e-31j), (2.974470498284723e-16-2.9582283945787943e-31j), (-0.03350041368950797+0j)]


How do the numerical and analytical result for Hellman-Feynman compare? Can you improve the agreement? 

Are the Pulay terms negligible?

In [14]:
print("Hellman-Feynman evaluated through finite differences {}".format(f_eval))
print("Hellman-Feynman evaluated analytically {}".format(f_op_hf_eval))
print("Full force evaluated analytically {}".format(f_op_eval))


Hellman-Feynman evaluated through finite differences [8.705475287582261e-16, -7.505472048578799e-16, -0.011280097482221882, -8.705475287582253e-16, 7.505472048578793e-16, 0.011280097367202285]
Hellman-Feynman evaluated analytically [0, 0, -0.0059691705428689245, 0, 0, 0.0059691705428693235]
Full force evaluated analytically [0, 0, 0.03350041368950822, 0, 0, -0.03350041368950797]


Most qchem packages have a method to perform analytical gradients without going over operators. For this system, a CASSCF(6,4) calculation is equivalent to FCI and we can obtain the analytical gradient in a different way.

In [15]:
fci = mcscf.CASSCF(mf, 6, 4).run()
fci.nuc_grad_method().kernel()

CASSCF energy = -7.87845365227714
CASCI E = -7.87845365227714  E(CI) = -9.01240481853429  S^2 = 0.0000000
--------------- CASSCF gradients ---------------
         x                y                z
0 Li    -0.0000000000    -0.0000000000     0.0335004137
1 H     0.0000000000     0.0000000000    -0.0335004137
----------------------------------------------


array([[-5.95066413e-18, -6.30267332e-18,  3.35004137e-02],
       [ 5.95066413e-18,  6.30267332e-18, -3.35004137e-02]])

# Difference with and without Pulay terms
The Pulay terms stem from the fact that Gaussian basis functions are centered at the nucle. In principle, the Pulay terms should disappear when the basis size approaches the basis set limit. We'll use correlation consistent basis functions to systematically increase the basis size. We can no longer use FCI for the expectation values here and will construct the RHF rdm to do that. The calculation of the integrals for the derivatives might take a while.

In [16]:
for basis in ['sto-3g', '6-31g', '6-311g']:
    mymol = pyscf.gto.M(
        atom = '''Li 0 0 0; H 0 0 1.4''',
        basis = basis)

    mymol.build()
    mf = pyscf.scf.RHF(mymol)
    mf.kernel()
    molecule = of.MolecularData(geometry=mymol.atom, basis=mymol.basis,
                                charge=0, multiplicity=1)
    molecule = run_pyscf(molecule)
    oei, tei = molecule.get_integrals()
    f_op = gradient_mo_operator(mymol, mf.mo_coeff, oei, tei)
    f_op_hf = gradient_mo_operator(mymol, mf.mo_coeff, oei, tei, with_pulay=False)
    opdm = np.diag([1] * mymol.nelectron + [0] * (2 * mymol.nao - mymol.nelectron))
    tpdm = 2 * of.wedge(opdm, opdm, (1, 1), (1, 1))
    rdms = of.InteractionRDM(opdm, tpdm)
    vector_expectation = np.vectorize(rdms.expectation)
    f_op_eval = vector_expectation(f_op).real
    f_op_hf_eval = vector_expectation(f_op_hf).real
    print(f"For {mymol.nao} spatial orbitals, the pulay forces have norm { np.linalg.norm(f_op_eval-f_op_hf_eval)}")

converged SCF energy = -7.8605386610207
For 6 spatial orbitals, the pulay forces have norm 0.06958084721938283
converged SCF energy = -7.97072364400739
For 11 spatial orbitals, the pulay forces have norm 0.036581788498768995
converged SCF energy = -7.97804025395656
For 16 spatial orbitals, the pulay forces have norm 0.041024551214650797


What is your conclusion? 

Extra excercise: Apply one of the Izmaylov group's grouping methods to see how the performance is for forces.